Run a Subset of the Full Experiments by Uploading Locations to Postgres
-------------------------------------------------------------
Danger: running this notebook clears the current database.


The first step to run the experiments is to upload locations from the data directory to Postgres.
Worker nodes will grab one location at a time and run them with Historia.

In [1]:
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
val path = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._


userhome: String = "/root"
jniPath: String = "/root/software/z3/build"
newPath: Array[String] = Array(
  "/root/software/z3/build",
  "/usr/lib/",
  "/usr/java/packages/lib",
  "/usr/lib64",
  "/lib64",
  "/lib",
  "/usr/lib"
)
res0_3: String = "/usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib"
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             


import plotly._, element._, layout._, Plotly._

import ujson.Value

import sys.process._


In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.{Driver,PickleSpec,Action,RunConfig,ExpTag}
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.{ExperimentsDb,BounderUtil}
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.{GetResult,SQLActionBuilder}
import scala.concurrent.Await
import edu.colorado.plv.bounder.lifestate.{SpecSpace,LifecycleSpec, SDialog, SAsyncTask,FragmentGetActivityNullSpec}
import scala.collection.parallel.CollectionConverters.IterableIsParallelizable



Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
//var android_home = android_home_possible.find(p => File(p).exists()).get             
//BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))
// System.setProperty("user.dir", s"${System.getProperty("user.home")}/data");
val expDir = File("/home/notebooks/dataSubset")
val baseDir = File("/home/historia_generalizability")

java.library.path set to: /root/software/z3/build:/usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.{Driver,PickleSpec,Action,RunConfig,ExpTag}

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.{ExperimentsDb,BounderUtil}

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.{GetResult,SQLActionBuilder}

import scala.concurrent.Await

import edu.colorado.plv.bounder.lifestate.{SpecSpace,LifecycleSpec, SDialog, SAsyncTask,FragmentGetActivityNullSpec}

import scala.collection.parallel.CollectionConverters.IterableIsParallelizable




android_home_possible: List[String] = List(
  "/root/Library/Android/sdk",
  "/root/Android/Sdk"
)
expDir: File = /home/notebooks/dataSubset
baseDir: File = /home/historia_generalizability

In [3]:
val benchmarkList = "benchmark-list-subset.txt"
object ExpConfig{
    val excludeFromProguardCheck = List("ch.blinkenlights.android.vanilla", "com.darshancomputing.BatteryIndicatorPro") // apps that break the tool I am using for proguard check but don't use proguard (checked manually)
    def apkAndFilt(file:File) = {
        val path = file.toString
        val spliton = if(path.contains("fdroid")) "fdroid" else "oss"
        val pkg = path.split(spliton)(1).split("/")
        s"${path}:${pkg.tail.head}"
    }
    val apkList = baseDir.glob("**/*.apk").toList.filter{pathF =>
        val path = pathF.toString // filter out the duplicate APKs for connectbot and antennapod
        !(path.contains("de.danoeh.antennapod") && path.contains("app-play")) && !(path.contains("org.connectbot") && path.contains("app-google"))}
        /*moved to another dir, comment out for speed*///.filter{a => excludeFromProguardCheck.exists(x => a.toString.contains(x)) || !Driver.detectProguard(a.toString)} // remove apps that use proguard (we can't distinguish app and framework)
    assert(apkList.exists(_.toString.contains("org.connectbot")))
    assert(apkList.exists(_.toString.contains("de.danoeh.antennapod")))
    val benchmarkListFile = File(benchmarkList)
    benchmarkListFile.overwrite(apkList.map(apkAndFilt(_)).mkString("\n"))
    
}
// Apps that use proguard (should be empty, moved to another folder)
ExpConfig.apkList.size

benchmarkList: String = "benchmark-list-subset.txt"
defined object ExpConfig
res2_2: Int = 47

In [4]:
object CountEach{
    def appName(cfg:File):String = {
        val split = cfg.toString.split("/")
        split.reverse.tail.head
    }
    def groupByApp(cfgs:List[File]):Map[String,List[File]] = cfgs.groupBy{appName}
    val cfgs = expDir.glob("**/*.cfg").map(f => read[RunConfig](f.contentAsString())).toSet
    val disallows = expDir.glob("**/Disallow*.json").toList.groupBy{f => 
        val spec = read[RunConfig](f.contentAsString).specSet
        val disSpec = spec.getDisallowSpecSet
        assert(disSpec.size == 1, "should only have one disallow spec per config")
        disSpec.head.target.identitySignature
    }
    val disallowByApp = disallows.map{ case (dis,cfgs) => (dis, groupByApp(cfgs))}
    val derefCallin = expDir.glob("**/SensitiveDerefCallinCaused_*").toList.filter(cfg => !cfg.toString.contains(".ipynb_checkpoints") && !cfg.toString.endsWith(".result"))
    val derefCallinByApp = groupByApp(derefCallin)
    val derefFieldSync = expDir.glob("**/SensitiveDerefFieldCausedSynch_*").toList.filter(cfg => !cfg.toString.contains(".ipynb_checkpoints") && !cfg.toString.endsWith(".result"))
    val derefFieldSyncByApp = groupByApp(derefFieldSync)
    val derefFieldFinish = expDir.glob("**/SensitiveDerefFieldCausedFinish_*").toList.filter(cfg => !cfg.toString.contains(".ipynb_checkpoints") && !cfg.toString.endsWith(".result"))
    val derefFieldFinishByApp = groupByApp(derefFieldFinish)
    val allCfg = {
        expDir.glob("**/SensitiveDerefFieldCausedSynch_*").toSet ++ expDir.glob("**/Disallow*").toSet ++ expDir.glob("**/SensitiveDerefCallinCaused_*") ++  expDir.glob("**/SensitiveDerefFieldCausedFinish_*")
    }.filter(cfg => !cfg.toString.contains(".ipynb_checkpoints") && !cfg.toString.endsWith(".result"))
    
    val allCfgDeser = allCfg.flatMap{cfg => 
        try{
            Some(read[RunConfig](cfg.contentAsString()))
        }catch{
            case _:upickle.core.AbortException => None // ignore non-cfg files
        }
    }.toSet
}
println(s"disallows (combined): ${CountEach.disallows.size}")
println(s"all size: ${CountEach.allCfg.size}")
println(s"distinct configurations: ${CountEach.allCfgDeser.size}") //why do we have one more config than earlier?
println(s"deref from callin: ${CountEach.derefCallin.size}")
println(s"deref from field finish: ${CountEach.derefFieldFinish.size}")
println(s"deref from field sync: ${CountEach.derefFieldSync.size}")
//println(s"disallow kinds: ${CountEach.disallows.keys}")
println(s"dismiss callins: ${CountEach.disallows("I_CIEnter_Dialogdismiss").size}")
println(s"execute callins: ${CountEach.disallows("I_CIEnter_AsyncTaskexecute").size}")

CountEach.derefCallin


disallows (combined): 2
all size: 10
distinct configurations: 10
deref from callin: 6
deref from field finish: 0
deref from field sync: 2
dismiss callins: 1
execute callins: 1


defined object CountEach
res3_9: List[File] = List(
  /home/notebooks/dataSubset/dev.ukanth.ufirewall/SensitiveDerefCallinCaused_8.json,
  /home/notebooks/dataSubset/net.wigle.wigleandroid/SensitiveDerefCallinCaused_7.json,
  /home/notebooks/dataSubset/com.darshancomputing.BatteryIndicatorPro/SensitiveDerefCallinCaused_22.json,
  /home/notebooks/dataSubset/com.nutomic.syncthingandroid/SensitiveDerefCallinCaused_3.json,
  /home/notebooks/dataSubset/de.blinkt.openvpn/SensitiveDerefCallinCaused_9.json,
  /home/notebooks/dataSubset/de.blinkt.openvpn/SensitiveDerefCallinCaused_8.json
)

In [5]:
// Utilities for manipulating run configs

def splitConfig(cfg:File, maxJobsPerFile:Int) = {
    val cfgContent = read[RunConfig](cfg.contentAsString)
    val parentDir = cfg.parent
    val cfgName = cfg.name.split('.')(0)
    assert(cfg.name.split('.')(1) == "json", "Input should have ext json")
    val newCfg = cfgContent.initialQuery.grouped(maxJobsPerFile).zipWithIndex
    
    newCfg.foreach{
        case (iqs,ind) => 
            val outfname = s"${cfgName}_${ind}.json"
            println(s"index: ${ind} size: ${cfg.size} fname: ${outfname}")
            val outf = parentDir / outfname
            outf.overwrite(write[RunConfig](cfgContent.copy(initialQuery=iqs)))
    }
}

// update timeout in run config
def setTimeout(nt:Int) = {
    //val configs = expDir.glob("**/*.json").toList
    val configs = CountEach.allCfg
    assert(configs.forall(f => !f.toString.contains(".ipynb_checkpoints")), s"Delete dumb jupyter backup file")
    configs.foreach{cfgf =>
        //println(cfgf.toString)
        val cfg = read[RunConfig](cfgf.contentAsString)
        cfgf.delete()
        cfgf.overwrite(write[RunConfig](cfg.copy(timeLimit=nt)))
    }
//     write[RunConfig](.map{c => read[RunConfig](c.contentAsString)}.toList.head.copy(timeLimit = 180))
}
//splitConfig(File("/Users/shawnmeier/Documents/source/bounder/experiments/5fdroid5dev_sampderef/com.lyonbros/sample.json"),5)

val db = new ExperimentsDb(Some(path))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

def pauseJobs() = {
    val pauseJobsQ = sql"""update jobs set status='paused' where status='new';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def resumeJobs() = {
    val pauseJobsQ = sql"""update jobs set status='new' where status='paused';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def clearInputs() = {
    val clearInputsQ = sql"""delete from apks where apkname like 'jar_%' or apkname like 'spec_%';"""
    Await.result(db.db.run(clearInputsQ.as[StrRes]), 30 seconds)
    val clearInputsTableQ = sql"""delete from inputs;"""
    Await.result(db.db.run(clearInputsTableQ.as[StrRes]), 30 seconds)
}
def resetJobs() = {
    val resetJobsQ = sql"""update jobs set inputid=4, stderr='',stdout='',owner='',started=NULL,ended=NULL,status='new';"""
    Await.result(db.db.run(resetJobsQ.as[StrRes]), 30 seconds)
}
def clearTable(tableName:String) = {
    val clear = sql"""delete from #$tableName ;"""
    Await.result(db.db.run(clear.as[StrRes]), 30 seconds)
}
def clearApks() = {
    
}
def clearResults(){
    val clearResults = sql"""delete from results; delete from resultdata;"""
    Await.result(db.db.run(clearResults.as[StrRes]), 30 seconds)
}
def clearAll(){
    clearInputs()
    clearResults()
    clearTable("jobs")
    clearTable("apks")
}

//clearResults()
//pauseJobs()
//clearAll()
//resumeJobs()
//Await.result(db.db.run(sql"""select count(*) from results;""".as[Count]), 30 seconds)

Initializing database


defined function splitConfig
defined function setTimeout
db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@4b88f195
defined class Count
defined class StrRes
getCountResult: AnyRef with GetResult[Count] = <function1>
getStrResult: AnyRef with GetResult[StrRes] = <function1>
defined function pauseJobs
defined function resumeJobs
defined function clearInputs
defined function resetJobs
defined function clearTable
defined function clearApks
defined function clearResults
defined function clearAll

Upload needed files to database
-------------------------------

In [6]:
// --- clear existing results in database
clearAll()
setTimeout(900)

In [7]:

val jarPath = File(path)
val cfg = File(s"/home/bounder/specs/cfg.txt") //note: unused
// Upload empty spec run config and bounder jar
val emptyCfg = db.createConfig(
    cfg,
    jarPath,
    "Empty spec set"
)

// Upload data of APK files to database for experiments
def uploadApks() = {
    def appList = (expDir / ".." / benchmarkList).contentAsString.split("\n")
        .map{v => 
            val spl = v.split(":")
            val name = spl(0).replace(baseDir.toString,"")
            val path = spl(0)
            (name,path)
        }
    appList.foreach(a => db.uploadApk(a._1,File(a._2)))
}


// push experiment .json files to database
def pushExp() = {
    // val experiments = expDir.glob("**/*.cfg").toList
    val experiments =  CountEach.allCfg
    println(experiments.size)
    experiments.foreach{v => 
                        val cfg = read[RunConfig](v.contentAsString)
                        db.createJob(v, Some(write[ExpTag](cfg.tag)), emptyCfg)
    }
}
uploadApks()
pushExp()

10


jarPath: File = /home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar
cfg: File = /home/bounder/specs/cfg.txt
emptyCfg: Int = 6
defined function uploadApks
defined function pushExp

Bulk Job Tasks
==============

Uncomment a function call to perform an action like clearing all job data, pausing jobs or resuing jobs.

In [7]:
//db.downloadApk("/com.gpl.rpg.AndorsTrail/0.7.5/apk/com.gpl.rpg.AndorsTrail_46.apk", File("/Users/shawnmeier/Desktop/testDownload.apk"))
//clearAll()
//resumeJobs()
//clearResults()
//pauseJobs()